In [1]:
import os
import shutil
import numpy as np
import PIL
from PIL import Image
import pandas as pd

from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, load_img

# Setting the working directory
os.chdir(r'C:\Users\Atul\Desktop\python files')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Firstly we make the folder path.
os.makedirs(r"C:\Users\Atul\Desktop\python files\data")
os.makedirs(r"C:\Users\Atul\Desktop\python files\data\train")
os.makedirs(r"C:\Users\Atul\Desktop\python files\data\test\test")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\aravi\\Desktop\\python files\\data'

In [3]:
src = r"C:\Users\Atul\Desktop\Jupyter\10_categories"
dest = r"C:\Users\Atul\Desktop\Jupyter\data\train\train"
copyDirectory(src, dest)

FileNotFoundError: [Errno 2] No such file or directory: 'test_image.jpg'

In [4]:
import shutil
 
def copyDirectory(src, dest):
    try:
        shutil.copytree(src, dest)
    # Directories are the same
    except shutil.Error as e:
        print('Directory not copied. Error: %s' % e)
    # Any error saying that the directory doesn't exist
    except OSError as e:
        print('Directory not copied. Error: %s' % e)

In [5]:
src = r"C:\Users\Atul\Desktop\Jupyter\10_categories"
dest = r"C:\Users\Atul\Desktop\Jupyter\data\train\train"
copyDirectory(src, dest)

In [6]:
#os.makedirs(r"C:\Users\Atul\Desktop\Jupyter\data\train\validation")

In [5]:
import os

inputpath = r"C:\Users\Atul\Desktop\python files\data\train\train"
outputpath = r"C:\Users\Atul\Desktop\python files\data\train\validation"

for dirpath, dirnames, filenames in os.walk(inputpath):
    word =dirpath[len(inputpath):]
    
    structure = outputpath +word
    if not os.path.isdir(structure):
        os.mkdir(structure)
    else:
        print("Folder does already exits!")

Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!
Folder does already exits!


In [7]:

base_dir = 'C:/Users/Atul/Desktop/python files/data'

trn = base_dir + "/train/train/"
dirName = os.listdir(trn)

for x in dirName:
    print(x)
    sourceN = base_dir + "/train/train/"+ x +"/"
    destN = base_dir + "/train/validation/"+ x

    filesN = os.listdir(sourceN)
    
    for f in filesN:
        if np.random.rand(1) < 0.2:
            shutil.move(sourceN + '/'+ f, destN + '/'+ f)
        
    print(len(os.listdir(sourceN)))
    print(len(os.listdir(destN)))


airplanes
278
522
BACKGROUND_Google
156
312
bonsai
47
81
car_side
51
72
Faces
135
300
Faces_easy
144
291
grand_piano
37
62
Leopards
71
129
Motorbikes
251
547
watch
128
111


In [8]:

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') # 'nearest' is kind of algorithm to fill pixel values while transformation

In [9]:
# Get VGG16 architecture from keras.applications
from keras.applications.vgg16 import VGG16,GlobalAveragePooling2D
from keras.models import Model

# downloading weights and convolution layers from vgg16
trained_model= VGG16(include_top=False,weights='imagenet') # include_top = False means it removes the MLP part of the N/W. So we get only the convolution part 

# defining mlp that needs to be appended to vgg16 convolution layers
x = trained_model.output
x = GlobalAveragePooling2D()(x) # we dont have a window size.. Do it on the whole thing
x = Dense(512,activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)

x = Dropout(0.4)(x)
output = Dense(10,activation='softmax')(x) # We can use either 2 neurons with softmax or one neuron with sigmoid
model= Model(inputs=trained_model.input,outputs=output)


In [10]:

for layer in trained_model.layers:
    layer.trainable=False 
    
#compiling
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=adam)

In [11]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Define train and validation generators
This is extremely useful when the samples are arranged in a structure, where all samples corresponding to one class are in a single folder. And all such folders corresponding to train/validation are in a parent folder. 

In [12]:
batch_size = 32

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# This is the augmentation configuration we will use for testing:
# Only rescaling. Other transformations are not required for testing. Duh!
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(  
        'data/train/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels
# save to directory can also be given as an argument here

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/train/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 1297 images belonging to 10 classes.
Found 2427 images belonging to 10 classes.


### Train the model

In [14]:

model.fit_generator(
        train_generator,
        steps_per_epoch=120//batch_size, # '//' in python returns only the quotient
        epochs=20,
        validation_data=validation_generator,
        validation_steps=45//batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/20
3/3 [==============================] - 19s 6s/step - loss: 0.9326 - acc: 0.6875 - val_loss: 0.7286 - val_acc: 0.7812
Epoch 2/20
3/3 [==============================] - 18s 6s/step - loss: 0.8872 - acc: 0.7083 - val_loss: 0.6497 - val_acc: 0.8125
Epoch 3/20
3/3 [==============================] - 18s 6s/step - loss: 0.6731 - acc: 0.7917 - val_loss: 0.6446 - val_acc: 0.7812
Epoch 4/20
3/3 [==============================] - 18s 6s/step - loss: 0.7122 - acc: 0.7604 - val_loss: 0.6378 - val_acc: 0.8125
Epoch 5/20
3/3 [==============================] - 18s 6s/step - loss: 0.7005 - acc: 0.7812 - val_loss: 0.5986 - val_acc: 0.8438
Epoch 6/20
3/3 [==============================] - 18s 6s/step - loss: 0.5362 - acc: 0.8438 - val_loss: 0.5009 - val_acc: 0.8438
Epoch 7/20
3/3 [==============================] - 18s 6s/step - loss: 0.6051 - acc: 0.8333 - val_loss: 0.4638 - val_acc: 0.8750
Epoch 8/20
3/3 [==============================] - 18s 6s/step - loss: 0.6447 - acc: 0.8125 - val_loss: 0

In [15]:

print(train_generator.class_indices) # This returns class labels of directories

{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}


In [16]:

df = pd.DataFrame.from_dict(train_generator.class_indices, orient = 'index')

### Test the model

In [17]:
os.getcwd()

'C:\\Users\\aravi\\Desktop\\python files'

In [18]:
test_generator = test_datagen.flow_from_directory(
        'data/test/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 1 images belonging to 1 classes.


In [19]:
test_prob = model.predict_generator(test_generator, steps=2) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)
print(test_prob)

[0 0]
[[0.7539706  0.00230637 0.00888526 0.00203927 0.02210028 0.00179307
  0.00929745 0.00113137 0.13414292 0.06433338]
 [0.7539706  0.00230637 0.00888526 0.00203927 0.02210028 0.00179307
  0.00929745 0.00113137 0.13414292 0.06433338]]


In [20]:
# Check the corresponding filenames of the predictions
print(test_generator.filenames)

['test\\test_image.jpg']


In [21]:
# Class Name for final
df.index[test_pred_classes[0]]

'BACKGROUND_Google'